In [7]:
file_dir = "../ariel_results/per_function_timing/"

import os, re, pandas as pd

TIMING_RE = re.compile(
    r"^\s*(?:-\s*)*"                 # leading “- ” blocks
    r"(?P<name>[^:]+?)\s+"           # function name
    r"(?:Took|took):\s+"             # “took:” (any case)
    r"(?P<secs>[0-9.eE+-]+)s",
    re.IGNORECASE,
)

def _parse(log: str) -> pd.DataFrame:
    rows, cur_tree = [], -1
    for line in log.splitlines():
        if "Selecting Bootstrapped Samples Took" in line:
            cur_tree += 1
        m = TIMING_RE.match(line)
        if m and cur_tree >= 0:
            rows.append(
                {
                    "tree": cur_tree + 1,
                    "function": m.group("name").strip(),
                    "time_s": float(m.group("secs")),
                }
            )
    return pd.DataFrame(rows)

# ── NEW: desired column order & renames ────────────────────────────────
ORDER = [
    "Selecting Bootstrapped Samples",
    "Initialization of FindBestCondOblique",
    "SampleProjection",
    "ApplyProjection",
    "Bucket Allocation & Initialization=0",
    "Filling & Finalizing the Buckets",
    "SortFeature",
    "ScanSplits",
    "Post-processing after Training all Trees",
    "EvaluateProjection",
    "FillExampleBucketSet (next 3 calls)",
]
RENAMES = {
    "Post-processing after Train": "Post-processing after Training all Trees",
    "FillExampleBucketSet (calls 3 above)": "FillExampleBucketSet (next 3 calls)",
}

def tree_table(log: str) -> pd.DataFrame:
    df = _parse(log)
    long = df.groupby(["tree", "function"], as_index=False)["time_s"].sum()

    wide = (
        long.pivot(index="tree", columns="function", values="time_s")
            .rename(columns=RENAMES)          # make names match ORDER list
            .fillna(0.0)
    )

    # ensure every requested column exists, then apply the order
    wide = wide.reindex(columns=ORDER, fill_value=0.0)

    # make “tree” a visible column instead of the index
    wide = wide.reset_index()

    return wide


In [4]:
raw_results = """
Selecting Bootstrapped Samples Took: 0.0546548s

 - Initialization of FindBestCondOblique Took: 0.0024063s


 - SampleProjection took: 7.447e-06s
 - ApplyProjection took: 0.00337401s

 - - Bucket Allocation & Initialization=0 took: 0.00177985s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0531089s
 - FillExampleBucketSet (calls 3 above) took: 0.0555681s

 - ScanSplits took: 0.00978284s

 - EvaluateProjection took: 0.0656653s



DecisionTree::Train() alone Took: 0.0840052s

I0000 00:00:1749496847.903380    5646 random_forest.cc:980] Train tree 1/20 [index:0 total:0.14s tree:0.14s]

Selecting Bootstrapped Samples Took: 0.0550178s

 - Initialization of FindBestCondOblique Took: 0.0023309s


 - SampleProjection took: 3.558e-06s
 - ApplyProjection took: 0.00510144s

 - - Bucket Allocation & Initialization=0 took: 0.00355848s
 - - Filling & Finalizing the Buckets took: 1.02e-07s
 - - SortFeature took: 0.127455s
 - FillExampleBucketSet (calls 3 above) took: 0.132506s

 - ScanSplits took: 0.0158169s

 - EvaluateProjection took: 0.148969s



DecisionTree::Train() alone Took: 0.16585s


Selecting Bootstrapped Samples Took: 0.052119s

 - Initialization of FindBestCondOblique Took: 0.00509593s


 - SampleProjection took: 1.012e-05s
 - ApplyProjection took: 0.00806058s

 - - Bucket Allocation & Initialization=0 took: 0.00369441s
 - - Filling & Finalizing the Buckets took: 6.1e-08s
 - - SortFeature took: 0.128416s
 - FillExampleBucketSet (calls 3 above) took: 0.133551s

 - ScanSplits took: 0.0339502s

 - EvaluateProjection took: 0.168155s



DecisionTree::Train() alone Took: 0.201746s


Selecting Bootstrapped Samples Took: 0.130735s

 - Initialization of FindBestCondOblique Took: 0.00230939s


 - SampleProjection took: 3.539e-06s
 - ApplyProjection took: 0.00343783s

 - - Bucket Allocation & Initialization=0 took: 0.00175805s
 - - Filling & Finalizing the Buckets took: 3.9e-08s
 - - SortFeature took: 0.0530352s
 - FillExampleBucketSet (calls 3 above) took: 0.0554698s

 - ScanSplits took: 0.00992014s

 - EvaluateProjection took: 0.0657019s



DecisionTree::Train() alone Took: 0.080589s


Selecting Bootstrapped Samples Took: 0.0526445s

 - Initialization of FindBestCondOblique Took: 0.00229882s


 - SampleProjection took: 5.303e-06s
 - ApplyProjection took: 0.00340606s

 - - Bucket Allocation & Initialization=0 took: 0.00179493s
 - - Filling & Finalizing the Buckets took: 2.5e-08s
 - - SortFeature took: 0.112302s
 - FillExampleBucketSet (calls 3 above) took: 0.114825s

 - ScanSplits took: 0.0341362s

 - EvaluateProjection took: 0.149338s



DecisionTree::Train() alone Took: 0.180214s


Selecting Bootstrapped Samples Took: 0.131625s

 - Initialization of FindBestCondOblique Took: 0.00514279s


 - SampleProjection took: 1.8675e-05s
 - ApplyProjection took: 0.00794652s

 - - Bucket Allocation & Initialization=0 took: 0.00371142s
 - - Filling & Finalizing the Buckets took: 8e-08s
 - - SortFeature took: 0.128939s
 - FillExampleBucketSet (calls 3 above) took: 0.134077s

 - ScanSplits took: 0.0338926s

 - EvaluateProjection took: 0.168625s



DecisionTree::Train() alone Took: 0.205183s


Selecting Bootstrapped Samples Took: 0.075812s

 - Initialization of FindBestCondOblique Took: 0.00233178s


 - SampleProjection took: 7.16e-06s
 - ApplyProjection took: 0.00343152s

 - - Bucket Allocation & Initialization=0 took: 0.00180023s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0552731s
 - FillExampleBucketSet (calls 3 above) took: 0.0577501s

 - ScanSplits took: 0.0097717s

 - EvaluateProjection took: 0.0678386s



DecisionTree::Train() alone Took: 0.0820184s


Selecting Bootstrapped Samples Took: 0.0544995s

 - Initialization of FindBestCondOblique Took: 0.00235147s


 - SampleProjection took: 5.275e-06s
 - ApplyProjection took: 0.00341382s

 - - Bucket Allocation & Initialization=0 took: 0.00181383s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0946974s
 - FillExampleBucketSet (calls 3 above) took: 0.0972586s

 - ScanSplits took: 0.0338615s

 - EvaluateProjection took: 0.131486s



DecisionTree::Train() alone Took: 0.154612s


Selecting Bootstrapped Samples Took: 0.130911s

 - Initialization of FindBestCondOblique Took: 0.00369635s


 - SampleProjection took: 1.1077e-05s
 - ApplyProjection took: 0.00339111s

 - - Bucket Allocation & Initialization=0 took: 0.0018152s
 - - Filling & Finalizing the Buckets took: 3.9e-08s
 - - SortFeature took: 0.0541748s
 - FillExampleBucketSet (calls 3 above) took: 0.0566769s

 - ScanSplits took: 0.0098007s

 - EvaluateProjection took: 0.0667903s



DecisionTree::Train() alone Took: 0.0850264s


Selecting Bootstrapped Samples Took: 0.0534186s

 - Initialization of FindBestCondOblique Took: 0.00227073s


 - SampleProjection took: 4.91e-06s
 - ApplyProjection took: 0.00342705s

 - - Bucket Allocation & Initialization=0 took: 0.00176094s
 - - Filling & Finalizing the Buckets took: 2.5e-08s
 - - SortFeature took: 0.0531844s
 - FillExampleBucketSet (calls 3 above) took: 0.0556726s

 - ScanSplits took: 0.0097948s

 - EvaluateProjection took: 0.0657804s



DecisionTree::Train() alone Took: 0.0831771s


Selecting Bootstrapped Samples Took: 0.0521215s

 - Initialization of FindBestCondOblique Took: 0.00229851s


 - SampleProjection took: 5.282e-06s
 - ApplyProjection took: 0.00341525s

 - - Bucket Allocation & Initialization=0 took: 0.00176654s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.0550842s
 - FillExampleBucketSet (calls 3 above) took: 0.0575293s

 - ScanSplits took: 0.00976635s

 - EvaluateProjection took: 0.0676066s



DecisionTree::Train() alone Took: 0.0816876s


Selecting Bootstrapped Samples Took: 0.0868018s

 - Initialization of FindBestCondOblique Took: 0.00500631s


 - SampleProjection took: 2.0508e-05s
 - ApplyProjection took: 0.00802943s

 - - Bucket Allocation & Initialization=0 took: 0.00362886s
 - - Filling & Finalizing the Buckets took: 8.7e-08s
 - - SortFeature took: 0.0869428s
 - FillExampleBucketSet (calls 3 above) took: 0.0920456s

 - ScanSplits took: 0.0098138s

 - EvaluateProjection took: 0.102505s



DecisionTree::Train() alone Took: 0.127132s


Selecting Bootstrapped Samples Took: 0.0545743s

 - Initialization of FindBestCondOblique Took: 0.00228227s


 - SampleProjection took: 5.854e-06s
 - ApplyProjection took: 0.00341923s

 - - Bucket Allocation & Initialization=0 took: 0.00181482s
 - - Filling & Finalizing the Buckets took: 3.9e-08s
 - - SortFeature took: 0.102314s
 - FillExampleBucketSet (calls 3 above) took: 0.104864s

 - ScanSplits took: 0.0301344s

 - EvaluateProjection took: 0.135372s



DecisionTree::Train() alone Took: 0.149595s


Selecting Bootstrapped Samples Took: 0.0545785s

 - Initialization of FindBestCondOblique Took: 0.00228648s


 - SampleProjection took: 6.681e-06s
 - ApplyProjection took: 0.00343037s

 - - Bucket Allocation & Initialization=0 took: 0.00178679s
 - - Filling & Finalizing the Buckets took: 3.6e-08s
 - - SortFeature took: 0.0552445s
 - FillExampleBucketSet (calls 3 above) took: 0.0577092s

 - ScanSplits took: 0.00981507s

 - EvaluateProjection took: 0.0678401s



DecisionTree::Train() alone Took: 0.0844989s


Selecting Bootstrapped Samples Took: 0.0541556s

 - Initialization of FindBestCondOblique Took: 0.0022712s


 - SampleProjection took: 4.826e-06s
 - ApplyProjection took: 0.00342206s

 - - Bucket Allocation & Initialization=0 took: 0.00179643s
 - - Filling & Finalizing the Buckets took: 2.6e-08s
 - - SortFeature took: 0.121676s
 - FillExampleBucketSet (calls 3 above) took: 0.124196s

 - ScanSplits took: 0.0302177s

 - EvaluateProjection took: 0.154763s



DecisionTree::Train() alone Took: 0.177662s


Selecting Bootstrapped Samples Took: 0.130708s

 - Initialization of FindBestCondOblique Took: 0.00382338s


 - SampleProjection took: 8.471e-06s
 - ApplyProjection took: 0.00340939s

 - - Bucket Allocation & Initialization=0 took: 0.00179381s
 - - Filling & Finalizing the Buckets took: 3.8e-08s
 - - SortFeature took: 0.0545795s
 - FillExampleBucketSet (calls 3 above) took: 0.0570462s

 - ScanSplits took: 0.00980307s

 - EvaluateProjection took: 0.0671635s



DecisionTree::Train() alone Took: 0.0899834s


Selecting Bootstrapped Samples Took: 0.054294s

 - Initialization of FindBestCondOblique Took: 0.00533608s


 - SampleProjection took: 2.0001e-05s
 - ApplyProjection took: 0.00801858s

 - - Bucket Allocation & Initialization=0 took: 0.0036596s
 - - Filling & Finalizing the Buckets took: 6e-08s
 - - SortFeature took: 0.105602s
 - FillExampleBucketSet (calls 3 above) took: 0.110743s

 - ScanSplits took: 0.00978906s

 - EvaluateProjection took: 0.121161s



DecisionTree::Train() alone Took: 0.144287s


Selecting Bootstrapped Samples Took: 0.0545251s

 - Initialization of FindBestCondOblique Took: 0.00228833s


 - SampleProjection took: 6.103e-06s
 - ApplyProjection took: 0.00342921s

 - - Bucket Allocation & Initialization=0 took: 0.00178293s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.0550935s
 - FillExampleBucketSet (calls 3 above) took: 0.057566s

 - ScanSplits took: 0.00979604s

 - EvaluateProjection took: 0.0676756s



DecisionTree::Train() alone Took: 0.0805455s


Selecting Bootstrapped Samples Took: 0.0536292s

 - Initialization of FindBestCondOblique Took: 0.00274065s


 - SampleProjection took: 7.224e-06s
 - ApplyProjection took: 0.00363588s

 - - Bucket Allocation & Initialization=0 took: 0.00199386s
 - - Filling & Finalizing the Buckets took: 1.04e-07s
 - - SortFeature took: 0.100034s
 - FillExampleBucketSet (calls 3 above) took: 0.103663s

 - ScanSplits took: 0.00976956s

 - EvaluateProjection took: 0.113769s



DecisionTree::Train() alone Took: 0.127407s


Selecting Bootstrapped Samples Took: 0.0528554s

 - Initialization of FindBestCondOblique Took: 0.0022799s


 - SampleProjection took: 6.673e-06s
 - ApplyProjection took: 0.00341229s

 - - Bucket Allocation & Initialization=0 took: 0.0017662s
 - - Filling & Finalizing the Buckets took: 3.7e-08s
 - - SortFeature took: 0.053222s
 - FillExampleBucketSet (calls 3 above) took: 0.0556634s

 - ScanSplits took: 0.00979212s

 - EvaluateProjection took: 0.0657669s



DecisionTree::Train() alone Took: 0.0786254s


Post-processing after Train Took: 0.00785465s

Training wall-time: 3.92807s
"""


In [8]:
tbl = tree_table(raw_results)
tbl.to_excel(os.path.join(file_dir, "tree_timings_512k_x_1024.xlsx"), index=False)
print(tbl.head())

function  tree  Selecting Bootstrapped Samples  \
0            1                        0.054655   
1            2                        0.055018   
2            3                        0.052119   
3            4                        0.130735   
4            5                        0.052644   

function  Initialization of FindBestCondOblique  SampleProjection  \
0                                      0.002406          0.000007   
1                                      0.002331          0.000004   
2                                      0.005096          0.000010   
3                                      0.002309          0.000004   
4                                      0.002299          0.000005   

function  ApplyProjection  Bucket Allocation & Initialization=0  \
0                0.003374                              0.001780   
1                0.005101                              0.003558   
2                0.008061                              0.003694   
3               